# Data Cleaning: InsideAirbnb

In [1]:
import pandas as pd
import os
# For Azure connection:
from azure.storage.blob import BlobServiceClient
from io import BytesIO
import io
import json

## Loading data from Blob Storage

In [2]:
# Azure Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=datalakestoragerentscape;AccountKey=w6Edf3np1A18vQIei31unvKWjGpyDUBqexvVauAwCeqOmnF1Bq7WsIEVplSEW+hT0q4ZzDi2KNh4+AStrOcI6g==;EndpointSuffix=core.windows.net"
container_name = "rentscape-blob"

In [3]:
try:
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    print("Connected to Azure Blob Storage.")
except Exception as e:
    print("Failed to connect to Azure Blob Storage:", e)

Connected to Azure Blob Storage.


In [4]:
blob_list = container_client.list_blobs()
print(container_name)
for blob in blob_list:
    print(f"+---{blob.name}")

rentscape-blob
+---barcelona_listings.csv
+---barcelona_reviews.csv
+---prague_listings.csv
+---prague_reviews.csv


In [5]:
# Function to load a CSV file from Azure Blob Storage into a Pandas DataFrame
def load_csv_from_blob(blob_path):
    blob_client = container_client.get_blob_client(blob_path)
    stream = BytesIO(blob_client.download_blob().readall())
    return pd.read_csv(stream)

## Cleaning listings data

We now load the `listings.csv` dataset from both cities, and will proceed to unify and clean the data according to the correct data types.

In [6]:
# Load the Prague and Barcelona listings
try:
    prg_listings_raw = load_csv_from_blob("prague_listings.csv")
    bcn_listings_raw = load_csv_from_blob("barcelona_listings.csv")
    print("Data loaded successfully!")
except Exception as e:
    print("Failed to load data from Azure Blob Storage:", e)

Data loaded successfully!


In [7]:
# Save the data sets we will work with separately, to preserve the raw data
prg_listings = prg_listings_raw
bcn_listings = bcn_listings_raw

In [9]:
prg_listings.head(2)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,23163,https://www.airbnb.com/rooms/23163,20240624031252,2024-06-24,previous scrape,Residence Karolina - KAROL12,"Unique and elegant apartment rental in Prague,...",NaN,https://a0.muscache.com/pictures/01bbe32c-3f13...,5282,...,4.97,4.93,4.86,NaN,t,70,69,0,0,0.19
1,23169,https://www.airbnb.com/rooms/23169,20240624031252,2024-06-24,city scrape,Residence Masna - Masna302,Masna studio offers a lot of space and privacy...,NaN,https://a0.muscache.com/pictures/b450cf2a-8561...,5282,...,4.86,4.97,4.69,NaN,t,70,69,0,0,0.70


In [13]:
prg_listings.columns.tolist()

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

In [14]:
unique_neighbourhoods = prg_listings["neighbourhood_cleansed"].unique().tolist()
unique_neighbourhoods

['Praha 1',
 'Praha 2',
 'Praha 3',
 'Praha 5',
 'Praha 8',
 'Praha 15',
 'Dolní Měcholupy',
 'Praha 9',
 'Praha 7',
 'Praha 10',
 'Praha 6',
 'Praha 4',
 'Praha 14',
 'Praha 13',
 'Velká Chuchle',
 'Kunratice',
 'Zličín',
 'Dubeč',
 'Zbraslav',
 'Petrovice',
 'Praha 12',
 'Praha 11',
 'Praha 21',
 'Praha 16',
 'Praha 17',
 'Šeberov',
 'Klánovice',
 'Štěrboholy',
 'Slivenec',
 'Újezd',
 'Ďáblice',
 'Dolní Počernice',
 'Praha 18',
 'Praha 20',
 'Libuš',
 'Řeporyje',
 'Březiněves',
 'Nebušice',
 'Satalice',
 'Praha 22',
 'Troja',
 'Dolní Chabry',
 'Čakovice',
 'Praha 19',
 'Lipence',
 'Lysolaje',
 'Vinoř',
 'Nedvězí',
 'Koloděje',
 'Přední Kopanina',
 'Suchdol',
 'Kolovraty']

In [15]:
unique_neighbourhoods = bcn_listings["neighbourhood_cleansed"].unique().tolist()
unique_neighbourhoods

['la Sagrada Família',
 'el Besòs i el Maresme',
 "el Camp d'en Grassot i Gràcia Nova",
 'el Barri Gòtic',
 'Sant Pere, Santa Caterina i la Ribera',
 'la Barceloneta',
 "la Dreta de l'Eixample",
 "el Camp de l'Arpa del Clot",
 'Sant Antoni',
 'el Poblenou',
 'la Vila Olímpica del Poblenou',
 'les Corts',
 'la Vila de Gràcia',
 'Vallcarca i els Penitents',
 'el Raval',
 'la Font de la Guatlla',
 'el Parc i la Llacuna del Poblenou',
 'el Fort Pienc',
 "la Nova Esquerra de l'Eixample",
 'el Clot',
 'el Poble Sec',
 'Diagonal Mar i el Front Marítim del Poblenou',
 'el Coll',
 'Sants',
 'Pedralbes',
 'el Guinardó',
 'el Putxet i el Farró',
 "l'Antiga Esquerra de l'Eixample",
 'Sant Gervasi - Galvany',
 'Sants - Badal',
 'el Baix Guinardó',
 'el Congrés i els Indians',
 'Navas',
 'la Bordeta',
 'Sant Martí de Provençals',
 'la Maternitat i Sant Ramon',
 'Sarrià',
 'la Prosperitat',
 'el Turó de la Peira',
 'Provençals del Poblenou',
 "la Font d'en Fargues",
 'el Carmel',
 'Hostafrancs',
 'la

In [10]:
# Get the column names as a list
prg_column_names = prg_listings.columns.tolist()
bcn_column_names = bcn_listings.columns.tolist()

# Ensure that both data sets contain the same columns
assert prg_column_names == bcn_column_names, "The column names or order do not match!"
print("Column names and order match exactly!")

Column names and order match exactly!


In [11]:
# Create a DataFrame with column names and their data types
prg_columns_info = pd.DataFrame({
    'Column Name': prg_listings.columns,
    'Data Type': prg_listings.dtypes
}).reset_index(drop=True)

# Display the DataFrame
prg_columns_info

,Column Name,Data Type
0,id,int64
1,listing_url,object
2,scrape_id,int64
3,last_scraped,object
4,source,object
...,...,...
70,calculated_host_listings_count,int64
71,calculated_host_listings_count_entire_homes,int64
72,calculated_host_listings_count_private_rooms,int64
73,calculated_host_listings_count_shared_rooms,int64


The following points cover the cleaning tasks that must be undertaken, based on the above assessment:
- **Dates**: Convert all date fields (e.g., `last_scraped`) from string format to proper date type.
- **IDs**: Ensure that all ID fields (e.g., `host_id`) are cast to integer type.
- **Count Variables**: Verify that all count-related fields (e.g., `number_of_reviews`) are cast to integer type.
- **Decimal Numerical Values**: Ensure that all decimal values (e.g., `review_scores_rating`) are cast to float type.
- `price`: Interpret the string structure, remove unnecessary characters and cast to float type.
- **Boolean Values**: Convert all boolean fields (e.g., `instant_bookable`) to integer type (0 or 1).
- Add a `city` column which contains the name of the city of the dataset, to allow identification after merging both datasets.

This can be achieved by consolidating all tasks into a single function that can be applied to each dataset, as they share an identical structure.

In [12]:
container_name = "bnpapi-rentscape-blob"

try:
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    print("Connected to Azure Blob Storage.")
except Exception as e:
    print("Failed to connect to Azure Blob Storage:", e)

Connected to Azure Blob Storage.


In [13]:
blob_list = container_client.list_blobs()
print(container_name)
for blob in blob_list:
    print(f"+---{blob.name}")

bnpapi-rentscape-blob
+---conversion_rates.csv
+---conversion_rates.json


In [14]:
def clean_listings(df, name):
    print(f"Dataset: {name} -------------")
    
    # Cast date columns from string to datetime
    date_columns = ['last_scraped', 'host_since', 'first_review', 'last_review', 'calendar_last_scraped']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    print("- Date columns cast to date type")
    
    # Cast ID columns to integer
    id_columns = ['id', 'scrape_id', 'host_id']
    for col in id_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce', downcast='integer').fillna(0).astype(int)
    print("- ID columns cast to integer")
    
    # Cast count columns to integer
    count_columns = [
        'host_listings_count', 'host_total_listings_count', 'availability_30', 'availability_60', 
        'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm', 
        'number_of_reviews_l30d', 'calculated_host_listings_count', 
        'calculated_host_listings_count_entire_homes', 
        'calculated_host_listings_count_private_rooms', 
        'calculated_host_listings_count_shared_rooms'
    ]
    for col in count_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce', downcast='integer').fillna(0).astype(int)
    print("- Count columns cast to integer")
    
    # Cast decimal numerical values to float
    decimal_columns = [
        'latitude', 'longitude', 'review_scores_rating', 'review_scores_accuracy', 
        'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 
        'review_scores_location', 'review_scores_value', 'reviews_per_month'
    ]
    for col in decimal_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0).astype(float)
    print("- Decimal numerical columns cast to float")
    
    # Clean and convert price column to float
    if 'price' in df.columns:
        df['price'] = df['price'].astype(str)  # Ensure all values are strings
        df['price'] = df['price'].str.replace('$', '', regex=False)  # Remove dollar sign
        df['price'] = df['price'].str.replace(',', '', regex=False)  # Remove commas
        df['price'] = pd.to_numeric(df['price'], errors='coerce')   # Convert to float, keeping NaNs
    print("- Price column values cleaned and cast to float")

    # Cast boolean values to integer (0/1)
    boolean_columns = ['instant_bookable', 'has_availability', 'host_is_superhost', 'host_identity_verified', 'host_has_profile_pic']
    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.lower().map({'true': 1, 'false': 0}).fillna(0).astype(int)
    print("- Boolean values cast to integer")

    # Add a 'city' column with the value of 'name'
    df['city'] = name
    print(f"- Added 'city' column with value '{name}' for all rows")
    
    return df

In [15]:
# Apply the cleaning function to both datasets
prg_listings_cleaned = clean_listings(prg_listings, "Prague")
bcn_listings_cleaned = clean_listings(bcn_listings, "Barcelona")

Dataset: Prague -------------
- Date columns cast to date type
- ID columns cast to integer
- Count columns cast to integer
- Decimal numerical columns cast to float
- Price column values cleaned and cast to float
- Boolean values cast to integer
- Added 'city' column with value 'Prague' for all rows
Dataset: Barcelona -------------
- Date columns cast to date type
- ID columns cast to integer
- Count columns cast to integer
- Decimal numerical columns cast to float
- Price column values cleaned and cast to float
- Boolean values cast to integer
- Added 'city' column with value 'Barcelona' for all rows


The [InsideAirbnb documentation](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit?gid=1322284596#gid=1322284596) states the following regarding the variable `price`: _daily price in local currency. NOTE: the $ sign is a technical artifact of the export, please ignore it_. For this purpose, the extracted currency exchange value will be applied to the `price` variable of the Prague dataset, in order to unify both to euro.

In [16]:
# Display a sample of the updated dataframe
prg_listings_cleaned.head()['price']

0       NaN
1    2779.0
2    3135.0
3    5218.0
4    3366.0
Name: price, dtype: float64

In [17]:
# Function to load JSON from Azure Blob Storage
def load_json_from_blob(connection_string, container_name, blob_name):
    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    
    # Download the blob
    blob_client = container_client.get_blob_client(blob_name)
    download_stream = blob_client.download_blob()
    
    # Parse JSON content
    return json.loads(download_stream.readall())

# Load the JSON file
conversion_rates_json = load_json_from_blob(connection_string, container_name, "conversion_rates.json")

In [18]:
conversion_rates_json

[{'Date': '2024-11-28',
  'CZK_to_PLN': 0.1705,
  'EUR_to_PLN': 4.3085,
  'CZK_to_EUR': 0.0395729372}]

In [19]:
# Extract the 'CZK_to_EUR' value
czk_to_eur_rate = conversion_rates_json[0].get("CZK_to_EUR", None)

In [20]:
# Apply the exchange rate to prg_listings_cleaned
prg_listings_cleaned["price"] = prg_listings_cleaned["price"] * czk_to_eur_rate

# Display a sample of the updated dataframe
prg_listings_cleaned.head()['price']

0           NaN
1    109.973192
2    124.061158
3    206.491586
4    133.202507
Name: price, dtype: float64

In [21]:
def check_missing_values(df, name):
    total_rows, total_columns = df.shape  # Get the number of rows and columns
    total_entries = df.size  # Total number of elements (rows * columns)
    missing_values = df.isnull().sum()
    total_missing = missing_values.sum()  # Total missing values across the entire dataset
    
    print(f"Missing values summary for dataset: {name} -------------")
    print(f"Shape: {total_rows} rows × {total_columns} columns")
    print(missing_values[missing_values > 0])
    print(f"Total missing values: {total_missing}/{total_entries} ({(total_missing / total_entries) * 100:.2f}%)\n")

# Apply to both datasets
check_missing_values(prg_listings_cleaned, "Prague")
check_missing_values(bcn_listings_cleaned, "Barcelona")

Missing values summary for dataset: Prague -------------
Shape: 9066 rows × 76 columns
description                      260
neighborhood_overview           4364
host_location                   2039
host_about                      3686
host_response_time               461
host_response_rate               461
host_acceptance_rate             268
host_neighbourhood               644
neighbourhood                   4364
neighbourhood_group_cleansed    9066
bathrooms                        620
bathrooms_text                    14
bedrooms                         100
beds                             630
price                            645
calendar_updated                9066
first_review                     784
last_review                      784
license                         9066
dtype: int64
Total missing values: 47322/689016 (6.87%)

Missing values summary for dataset: Barcelona -------------
Shape: 19482 rows × 76 columns
description                817
neighborhood_overview     9444


## Cleaning reviews data

In [22]:
# Reconnect to InsideAirbnb Blob Storage
container_name = "rentscape-blob"

try:
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    print("Connected to Azure Blob Storage.")
except Exception as e:
    print("Failed to connect to Azure Blob Storage:", e)

Connected to Azure Blob Storage.


In [23]:
# Load the Prague and Barcelona reviews
container_name = "rentscape-blob"

try:
    prg_reviews_raw = load_csv_from_blob("prague_reviews.csv")
    bcn_reviews_raw = load_csv_from_blob("barcelona_reviews.csv")
    print("Data loaded successfully!")
except Exception as e:
    print("Failed to load data from Azure Blob Storage:", e)

Data loaded successfully!


In [24]:
# Save the data sets we will work with separately, to preserve the raw data
prg_reviews = prg_reviews_raw
bcn_reviews = bcn_reviews_raw

In [25]:
prg_reviews.head(2)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,23163,101588,2010-09-20,227165,Nathan,Incredible apartment in an ideal location. The...
1,23163,157152,2010-12-22,286036,Hugh,"The apartment was huge, we felt like we were s..."


In [26]:
# Get the column names as a list
prg_column_names = prg_reviews.columns.tolist()
bcn_column_names = bcn_reviews.columns.tolist()

# Ensure that both data sets contain the same columns
assert prg_column_names == bcn_column_names, "The column names or order do not match!"
print("Column names and order match exactly!")

Column names and order match exactly!


The following points cover the cleaning tasks that must be undertaken, based on the above assessment:
- **IDs**: Ensure that all ID fields (`listing_id`, `id`, `reviewer_id`) are cast to integer type.
- **Dates**: Convert the `date` field from string format to proper date type.

This can be achieved by consolidating all tasks into a single function that can be applied to each dataset, as they share an identical structure.

In [27]:
def clean_reviews(df, name):
    print(f"Dataset: {name} -------------")
    
    # Cast date columns from string to datetime
    date_columns = ['date']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    print("- Date columns cast to date type")
    
    # Cast ID columns to integer
    id_columns = ['listing_id', 'id', 'reviewer_id']
    for col in id_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce', downcast='integer').fillna(0).astype(int)
    print("- ID columns cast to integer")
    
    # Optional: Handle missing values in `comments`
    if 'comments' in df.columns:
        df['comments'] = df['comments'].fillna("")
    print("- Missing values in comments handled (if applicable)")

     # Add a 'city' column with the value of 'name'
    df['city'] = name
    print(f"- Added 'city' column with value '{name}' for all rows")
    
    return df

In [28]:
# Apply the cleaning function to both datasets
prg_reviews_cleaned = clean_reviews(prg_reviews, "Prague")
bcn_reviews_cleaned = clean_reviews(bcn_reviews, "Barcelona")

Dataset: Prague -------------
- Date columns cast to date type
- ID columns cast to integer
- Missing values in comments handled (if applicable)
- Added 'city' column with value 'Prague' for all rows
Dataset: Barcelona -------------
- Date columns cast to date type
- ID columns cast to integer
- Missing values in comments handled (if applicable)
- Added 'city' column with value 'Barcelona' for all rows


In [29]:
# Apply missing value check to both datasets
check_missing_values(prg_reviews_cleaned, "Prague")
check_missing_values(bcn_reviews_cleaned, "Barcelona")

Missing values summary for dataset: Prague -------------
Shape: 651459 rows × 7 columns
Series([], dtype: int64)
Total missing values: 0/4560213 (0.00%)

Missing values summary for dataset: Barcelona -------------
Shape: 927474 rows × 7 columns
Series([], dtype: int64)
Total missing values: 0/6492318 (0.00%)



## Merge `listings` and `reviews` datasets

Merge the `listings` datasets by appending one dataframe after the other, with the column `city` serving for identification:

In [30]:
# Merge listings datasets
cities_listings = pd.concat([prg_listings_cleaned, bcn_listings_cleaned], ignore_index=True)
print("Merged listings datasets into 'cities_listings'.")

Merged listings datasets into 'cities_listings'.


In [31]:
# Count the occurrences of each unique value in the 'city' column
city_listings_counts = cities_listings['city'].value_counts()
city_listings_counts

city
Barcelona    19482
Prague        9066
Name: count, dtype: int64

Merge the `reviews` datasets by appending one dataframe after the other, with the column `city` serving for identification:

In [32]:
# Merge reviews datasets
cities_reviews = pd.concat([prg_reviews_cleaned, bcn_reviews_cleaned], ignore_index=True)
print("Merged reviews datasets into 'cities_reviews'.")

Merged reviews datasets into 'cities_reviews'.


In [33]:
# Count the occurrences of each unique value in the 'city' column
city_reviews_counts = cities_reviews['city'].value_counts()
city_reviews_counts

city
Barcelona    927474
Prague       651459
Name: count, dtype: int64

## Upload to Blob Storage

In [34]:
container_name = "cleansed-layer-airbnb"

# Function to upload a DataFrame to Azure Blob Storage
def upload_dataframe_to_blob(dataframe, container_name, blob_name, connection_string):
    # Convert the DataFrame to a CSV string
    csv_data = dataframe.to_csv(index=False)
    
    # Encode the CSV string to bytes
    csv_bytes = csv_data.encode('utf-8')
    
    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    
    # Get a blob client
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    
    # Upload the CSV data
    blob_client.upload_blob(csv_bytes, blob_type="BlockBlob", overwrite=True)
    print(f"Uploaded {blob_name} to container {container_name}.")

# File names for the blobs
listings_blob_name = "cities_listings.csv"
reviews_blob_name = "cities_reviews.csv"

# Upload the merged datasets
upload_dataframe_to_blob(cities_listings, container_name, listings_blob_name, connection_string)
upload_dataframe_to_blob(cities_reviews, container_name, reviews_blob_name, connection_string)

Uploaded cities_listings.csv to container cleansed-layer-airbnb.
Uploaded cities_reviews.csv to container cleansed-layer-airbnb.
